In [3]:
%reset

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import logging

Logging is one of the more missed libraries. It is a standard library. 

    The key benefit of having the logging API provided by a standard library module is that all Python modules can participate in logging, so your application log can include your own messages integrated with messages from third-party modules.

It is needed. 

* The guide explains what to do, but not why, and interface is not easy to link to actions or understand the right order of operations, missing all details. Interface is inconsistent in different objects (e.g. setLevel, baseconfig)

* The online documentation explains a few details, but there are not many examples of a real-world application (exemplar the fact that one of the seminal example cannot be launched multiple times). This is a lot of boilerplate code and less understanding.

* While the online documentation describes all objects and interfaces in the library, however Inline documentation is poor, see example getLogger, even if Logger is somewhat better

* The library is little introspective, in retrieving values and status from the different components. It is very difficolt to retrieve the format or the status of what is logging.


A description of the library is given by:

    This module defines functions and classes which implement a flexible event logging system for applications and libraries.
    [..]

    * Loggers expose the interface that application code directly uses.
    * Handlers send the log records (created by loggers) to the appropriate destination.
    * Filters provide a finer grained facility for determining which log records to output.
    * Formatters specify the layout of log records in the final output.

## Logging without loggers

Starting from obvious source https://docs.python.org/3/howto/logging.html most minimalistic example:

In [6]:
logging.basicConfig(level=logging.DEBUG)
logging.warning('This is a warning message') 

In this code I did not set a specific logger, but I am just calling messages by the generic ``logging`` module. There are a bunch of operations I can do in this way: I have set the level for which we have a log, and i implicitely chose to output on console (I didn't include a filename argument in calling ``.basicConfig``).

I can control output format: 

    logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

Here is a list of format codes which can be used, 
https://docs.python.org/3/library/logging.html#logrecord-attributes,
you can even modify colors or font formats (on supported terminals).

Note that trying to redefine the properties of a logging system (e.g. format or level) by calling basicconfig on an already configured logging module is not the suggested way, but also basicConfig documentation specifically says it has no effect if it was already called unless a ``force`` flag argument is used.
The flag seems to be necessary in this case.

Next cell logs 3 messages of level warning, error and debug,
``logging`` is sete to level ``WARNING``, i messaggi ``INFO`` non vengono stampati.

In [32]:
# test
logging.warning("This is a warning message")
logging.error("This is an error message")
logging.debug("This is a debug message")

ERROR:root:This is an error message
DEBUG:root:This is a debug message


In [33]:
logging.basicConfig??

Signature: logging.basicConfig(**kwargs)
Source:   
def basicConfig(**kwargs):
    """
    Do basic configuration for the logging system.

    This function does nothing if the root logger already has handlers
    configured, unless the keyword argument *force* is set to ``True``.
    It is a convenience method intended for use by simple scripts
    to do one-shot configuration of the logging package.

    The default behaviour is to create a StreamHandler which writes to
    sys.stderr, set a formatter using the BASIC_FORMAT format string, and
    add the handler to the root logger.

    A number of optional keyword arguments may be specified, which can alter
    the default behaviour.

    filename  Specifies that a FileHandler be created, using the specified
              filename, rather than a StreamHandler.
    filemode  Specifies the mode to open the file, if filename is specified
              (if filemode is unspecified, it defaults to 'a').
    format    Use the specified for

In the previous one, the second call to basicConfig overrode the first for the formattier. In second call ``logging.WARNING`` set the level of the logger (not showing the debug message).

This is a simple and acceptable way to use the library for a single logging channel:
I can use color and formatting codes in the format, decide to log on different output (to log on a file, pass arguments ``filename`` to log on file), as a global setting of ``Logging``, and also define new level names/value or redefine old ones (this is probably almost never necessary).

In [31]:
logging.warning("This is a warning message")
logging.error("This is an error message")
logging.debug("This is a debug message")

2022-05-28 00:57:57,902 WARNING This is a warning message
2022-05-28 00:57:57,902 ERROR This is an error message
2022-05-28 00:57:57,903 DEBUG This is a debug message


In [38]:
logging.basicConfig(format='%(asctime)s \033[1;31m%(levelname)s\033[1;0m %(message)s', force = True) #this applies to current level DEBUG, but doens't work
logging.getLogger().setLevel(logging.DEBUG) #level is not defined here rathere than on basicConfig
logging.warning("This is a warning message")

2022-05-29 12:20:56,913 WARNING This is a warning message


In [50]:
l = logging.getLogger()



In [51]:
reset_logger(logging.getLogger())
#logging.shutdown()

In [54]:
logging??


Type:        module
String form: <module 'logging' from 'c:\\Users\\kovor\\anaconda3\\envs\\folder_explorer\\lib\\logging\\__init__.py'>
File:        c:\users\kovor\anaconda3\envs\folder_explorer\lib\logging\__init__.py
Source:     
# Copyright 2001-2017 by Vinay Sajip. All Rights Reserved.
#
# Permission to use, copy, modify, and distribute this software and its
# documentation for any purpose and without fee is hereby granted,
# provided that the above copyright notice appear in all copies and that
# both that copyright notice and this permission notice appear in
# supporting documentation, and that the name of Vinay Sajip
# not be used in advertising or publicity pertaining to distribution
# of the software without specific, written prior permission.
# VINAY SAJIP DISCLAIMS ALL WARRANTIES WITH REGARD TO THIS SOFTWARE, INCLUDING
# ALL IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS. IN NO EVENT SHALL
# VINAY SAJIP BE LIABLE FOR ANY SPECIAL, INDIRECT OR CONSEQUENTIAL DAMAGES OR
# AN

In [58]:
from logs import start_logger
logging.shutdown()
start_logger()
l=logging.getLogger()
l.debug("debug")

2022-05-29 13:21:58,287 DEBUG debug


N:  logs
logger non defined, get one.
logger <RootLogger root (DEBUG)>, ID: 1690725819728


## Loggers without a name

Looking back at module documentation, ``Logging`` is based on one or more ``Logger``s. Each logger can have any number of ``Handlder``s, meaning output logging channels, each one corresponding to a output or to different settings.


The documentation of ``Logger`` says: 

    Note that Loggers are never instantiated directly, but always through the module-level function logging.getLogger(name). Multiple calls to getLogger() with the same name will always return a reference to the same Logger object.

which says:

    logging.getLogger(name=None)¶

    Return a logger with the specified name or, if name is None, return a logger which is the root logger of the hierarchy. [..]
    All calls to this function with a given name return the same logger instance. This means that logger instances never need to be passed between different parts of an application.

I can then repeat what was done in previous example on the ``Logger`` object returned by ``logging.getLogger()``.

In [19]:
logging.basicConfig(format='%(asctime)s \033[1;31m%(levelname)s\033[1;0m %(message)s', force = True) #this applies to current level DEBUG, but doens't work
logging.getLogger().setLevel(logging.DEBUG) #level is not defined here rathere than on basicConfig
logging.warning("This is a warning message")
logging.error("This is an error message")
logging.debug("This is a debug message")

2022-05-28 00:53:31,574 WARNING This is a warning message
2022-05-28 00:53:31,575 ERROR This is an error message
2022-05-28 00:53:31,575 DEBUG This is a debug message


The documentation is concise, but informative:

    Signature: logging.getLogger(name=None)
    Docstring:
    Return a logger with the specified name, creating it if necessary.

    If no name is specified, return the root logger.
    File:      c:\users\kovor\anaconda3\envs\folder_explorer\lib\logging\__init__.py
    Type:      function

It is evident that ``logging.getLogger`` is not a pointer to a "current logger", rather a string that identifies the logger by its unique name. When the function is called without passing a "name" argument, it returns always the same instance of logger (having default name ``"root"``).

Nel seguente esempio logger1 e logging agiscono sullo stesso logger, questo perchè logger1 era stato ottenuto chiamando getLogger senza argomenti. Posso quindi settare individualmente ad es.i livelli di log per logger1/logging e logger2 (o per ogni altro logger identificato da un nome). 

In [ ]:
logger1.setLevel(logging.INFO)

print("messaggi su logging:",flush=True)
logging.warning("This is a warning message")
logging.error("This is an error message")
logging.debug("This is a debug message")
print("messaggi su logger1:",flush=True)
logger1.warning("This is a warning message")
logger1.error("This is an error message")
logger1.debug("This is a debug message")
print("messaggi su logger2:",flush=True)
logger2.warning("This is a warning message")
logger2.error("This is an error message")
logger2.debug("This is a debug message")

messaggi su logging:


ERROR:root:This is an error message


messaggi su logger1:


ERROR:root:This is an error message


messaggi su logger2:


ERROR:__main__:This is an error message
DEBUG:__main__:This is a debug message


In ogni caso sembra che ogni chiamata a logging senza nome indirizza su un logger di note "root". Se inveve logger sono creati con nomi separati, il nome li identifica univocamente. 


There is also another difference between calling a logger with or without name, that is the type of logger returned.

In [18]:
print(logging.getLogger())
print(logging.getLogger(__name__))

<RootLogger root (WARNING)>
<Logger __main__ (WARNING)>


From respective docstrings:

In [26]:
l = logging.getLogger()
l?

Type:           RootLogger
String form:    <RootLogger root (WARNING)>
File:           c:\users\kovor\anaconda3\envs\folder_explorer\lib\logging\__init__.py
Docstring:     
A root logger is not that different to any other logger, except that
it must have a logging level and there is only one instance of it in
the hierarchy.
Init docstring: Initialize the logger with the name "root".


In [20]:
l = logging.getLogger(__name__)
l?

Type:           Logger
String form:    <Logger __main__ (WARNING)>
File:           c:\users\kovor\anaconda3\envs\folder_explorer\lib\logging\__init__.py
Docstring:     
Instances of the Logger class represent a single logging channel. A
"logging channel" indicates an area of an application. Exactly how an
"area" is defined is up to the application developer. Since an
application can have any number of areas, logging channels are identified
by a unique string. Application areas can be nested (e.g. an area
of "input processing" might include sub-areas "read CSV files", "read
XLS files" and "read Gnumeric files"). To cater for this natural nesting,
channel names are organized into a namespace hierarchy where levels are
separated by periods, much like the Java or Python package namespace. So
in the instance given above, channel names might be "input" for the upper
level, and "input.csv", "input.xls" and "input.gnu" for the sub-levels.
There is no arbitrary limit to the depth of nesting.
In

Calling a logger without a name is then a safe way to use ``logging``: everytime the function is called, returns the same `root` logger; any call to logging methods is broadcasted to this logger.

A single logger can then be accessed by any point of the program or subroutines and it is certainly identified from any entry point, and any change of level is safe; multiple formatters, output channel, or levels can be customized at level of handlers. 

In [14]:
logging.basicConfig()



logger.setLevel(lvl)


logger.getEffectiveLevel()
logging._Level
logging.addLevelName
logging.getLevelName
logging.getLogger
logging.Logger

LOGGER
addHandler(hdlr)
Adds the specified handler hdlr to this logger.

removeHandler(hdlr)
Removes the specified handler hdlr from this logger.

findCaller(stack_info=False)
Finds the caller’s source filename and line number. Returns the filename, line number, function name and stack information as a 4-element tuple. The stack information is returned as None unless stack_info is True.

handle(record)
Handles a record by passing it to all handlers associated with this logger and its ancestors (until a false value of propagate is found). This method is used for unpickled records received from a socket, as well as those created locally. Logger-level filtering is applied using filter().

getChild(suffix)

HANDLERS

setLevel(lvl)
Sets the threshold for this handler to lvl. Logging messages which are less severe than lvl will be ignored. When a handler is created, the level is set to NOTSET (which causes all messages to be processed).

See Logging Levels for a list of levels.

Changed in version 3.2: The lvl parameter now accepts a string representation of the level such as ‘INFO’ as an alternative to the integer constants such as INFO.

setFormatter(form)
Sets the Formatter for this handler to form.

close()
Tidy up any resources used by the handler. This version does no output but removes the handler from an internal list of handlers which is closed when shutdown() is called. Subclasses should ensure that this gets called from overridden close() methods.

handle(record)
Conditionally emits the specified logging record, depending on filters which may have been added to the handler. Wraps the actual emission of the record with acquisition/release of the I/O thread lock.





SyntaxError: invalid syntax (1374808952.py, line 15)

On the opposite, calling a logger with a name is a way to establish hirarchies of "logging channels" and manage them accordingly. For example, the suggested way of calling loggers passing ```__name__``` as name, can be used to mirror the hierarchy of source files inside a package (if I understand it right, this is not really explained in documentation). 

Note that if I chose to use the single logger approach, it is still possible to obtain a finer tuning for different "loggin channels" (set as different ```streams```) by manipulating the corresponding StreamHandlers (however it is not natural to customize behavior on the base of which file logging is called from).

## Avoiding traps

## Handlers

Considering what said above, this is a good point to start good-practices and avoiding traps, in order to begin logging!



However this textbook example, if launched multiple times add an handler every time:

In [ ]:
import logging

# create logger
logger = logging.getLogger()  #(__name__)  # indipendentemente sia chiamato con __name__ o senza.
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# add formatter to ch
ch.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))

# add ch to logger
logger.addHandler(ch)

# 'application' code
logger.debug('debug message')
logger.info('info message')
logger.warning('warn message')
logger.error('error message')
logger.critical('critical message')

DEBUG:root:debug message
2022-05-23 22:26:03,729 - root - DEBUG - debug message
INFO:root:info message
2022-05-23 22:26:03,731 - root - INFO - info message
2022-05-23 22:26:03,732 - root - WARNING - warn message
ERROR:root:error message
2022-05-23 22:26:03,733 - root - ERROR - error message
CRITICAL:root:critical message
2022-05-23 22:26:03,734 - root - CRITICAL - critical message


In [ ]:
print(logger)
print(ch)

<RootLogger root (DEBUG)>


NameError: name 'ch' is not defined

In [ ]:
logger.handlers

[<StreamHandler stderr (NOTSET)>, <StreamHandler stderr (DEBUG)>]

#### reset_logger

In [13]:
logging.getLogger()

<RootLogger root (DEBUG)>

In [ ]:
def reset_logger(logger):
    """remove all handlersr in place. A better alternative is probably `logging.shoutdown`."""
    for i in range(len(logger.handlers)):
        logger.handlers.pop()
        


In [ ]:
from logs import reset_logger
#reset_logger(logger)  # logger is my 


In [ ]:
reset_logger(logger)

NameError: name 'logger' is not defined

In [ ]:
logger.handlers

NameError: name 'logger' is not defined

#### start_logger

In [ ]:
# Questo era in dataIO.logger e trattava un problema che mi 
# avveniva spesso in passato su notebook in cui avevo definito handlers 
# multipli per loggare sia su file che su console

import logging


def start_logger(cfgfile=None,handlers=None):

    #if logger exists (this is based only in checking it has some handler) 
    # recover the current logger without adding handlers, if not load from file
    #  or if config file is not provided, uses default options for logger
        
    print("N: ",__name__) # qui mostra non serve perche sara start_logger    
    try:
        logger=logging.getLogger()
        print("logger %s existing"%logger)
    except:
        print("logger non existing.")
        
    logger=logging.getLogger()
    if len(logger.handlers) == 0:
        """not sure about original indentation here."""   
        if cfgfile is not None:
            logger=logging.config.fileConfig(cfgfile)
        else:
            logging.basicConfig(format='%(asctime)s %(message)s',force=True)  #set default     
            logger = logging.getLogger()
            #print('cfgfile missing, and no logger existing, not sure what happens now.')
            
    return logger


In [ ]:
# Qua si nota che anche con ogni possibile reset, il rosso resta..
reset_logger(logger)
logger = start_logger()
# 'application' code
logger.debug('debug message')
logger.info('info message')
logger.warning('warn message')
logger.error('error message')
logger.critical('critical message')

2022-05-23 22:12:18,720 debug message
2022-05-23 22:12:18,721 info message
2022-05-23 22:12:18,721 warn message
2022-05-23 22:12:18,722 error message
2022-05-23 22:12:18,722 critical message


N:  dataIO.logs
logger <RootLogger root (DEBUG)> existing


In [ ]:
from logs import start_logger

logger = start_logger()
# 'application' code
logger.debug('debug message')
logger.info('info message')
logger.warning('warn message')
logger.error('error message')
logger.critical('critical message')

DEBUG:root:debug message
INFO:root:info message
ERROR:root:error message
CRITICAL:root:critical message


N:  logs
logger <RootLogger root (DEBUG)> existing


In [ ]:
logger.handlers

[<StreamHandler stderr (NOTSET)>]

#### start_logger2

In [ ]:
#create two handlers file and console.

import logging

def start_logger2(logger = None):
    """Start logger with two handlers."""
    
    # create logger
    if logger is None:
        logger = logging.getLogger()
    
    logger.setLevel(logging.DEBUG)

    # create console handler and set level to debug
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    ch.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
    logger.addHandler(ch)

    # create file handler and set level to debug
    fh = logging.FileHandler('logging_tests.log')
    fh.setLevel(logging.DEBUG)
    fh.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
    logger.addHandler(fh)
    return logger

reset_logger(logger)
logger=start_logger2(logger)

# 'application' code
logger.debug('debug message')
logger.info('info message')
logger.warning('warn message')
logger.error('error message')
logger.critical('critical message')

AttributeError: 'NoneType' object has no attribute 'handlers'

Sicuramente configurare con conf

## Loggers with a name


So, why not to use a single logger, with multiple handlers which can be individually configured?

* without an entry point it is not sure how and if the logging environment was initialized. Adding handlers inside functions can lead to problems due to persistance of handlers that lead to repeated instances (unless checking if 
handlers already exist).

* you cannot easily identify where the logging was called from, as the logger is always the same.

* All calls, included from external routines which use the same nameless calls, share the logging system. You may want e.g. to intentionally set a hierarchy of behaviours, for example if I am working on a code which contains and relies on a library under development for another project, I may want to silence the messages from the cointained function setting its level to WARNING, and keep INFO on the container software. 

But what is usually suggested is instead to call ``getlogger`` with argumnent ``__name__``. This will result in the creation (or use) of one single logger for each function.

-------------------------------------------

### The `__name__` special variable

Forse qui e' il momento di capire quello che dice la guida sul nome dei logger e 
``__name__``. 

This import same code as previous cell from an external file, note that function names are correctly identified:

In [ ]:

#sys.path.append('pyXTel\source\dataIO\\')
from logs import logging_function as lf
    

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
print('\nlogger without name: %s ID: %s'%(logger,id(logger)),flush=True)
logger.warning('Warning raised by logger')
lf()

logger = logging.getLogger(__name__)
print("\n--\nlogger %s called from name %s, ID: %s" %(logger,__name__,id(logger)),flush=True)
logger.warning('Warning raised by logger')
lf()



logger without name: <RootLogger root (DEBUG)> ID: 2014591529456


2022-05-24 17:45:12,105 WARNING Warning raised by logger



In Logging Function  <function logging_function at 0x000001D513ECAD30>
logging:  <module 'logging' from 'c:\\Users\\kovor\\anaconda3\\envs\\folder_explorer\\lib\\logging\\__init__.py'>
logger without name: <RootLogger root (DEBUG)> ID: 2014591529456


2022-05-24 17:45:12,106 WARNING Warning raised by logger


logger <Logger logs (DEBUG)> called with name logs, ID: 2014663503248


...

2022-05-24 17:45:12,108 WARNING Warning raised by logger



In Logging Function  <function logging_function at 0x000001D513ECAD30>
logging:  <module 'logging' from 'c:\\Users\\kovor\\anaconda3\\envs\\folder_explorer\\lib\\logging\\__init__.py'>
logger without name: <RootLogger root (DEBUG)> ID: 2014591529456


2022-05-24 17:45:12,109 WARNING Warning raised by logger


logger <Logger logs (DEBUG)> called with name logs, ID: 2014663503248


2022-05-24 17:45:12,110 WARNING Warning raised by logger


-------------------------------------------

### Loggers with ``__name__`` inside functions

Note that calling a logger inside a function from a cell, gives a single root ``__name__ = "__main__"`` logger and the functions are not recognized, e.g.:

In [ ]:
# dava errore mortale, ora funziona
# Inizialmente pensato fosse dovuto a reset e autoreload all'inizio, si è poi scoperto 
#    che era dovuto ad un max di profondità in cui si puo' suppongo eseguire un notebook
#  (8 livelli?).
#  Vedi esempio printcane in dataIO 
# 

import logging

def logging_function():
    """A function which prints and log info. No return value"""
    
    print("\nIn Logging Function ",logging_function)
    print("logging: ",logging,flush=True)
    logger = logging.getLogger()
    print('logger without name: %s ID: %s'%(logger,id(logger)),flush=True)
    logger.warning('Warning raised by logger')
    logger = logging.getLogger(__name__)
    print("logger %s called with name \033[1;31m%s\033[1;0m, ID: %s" %(logger,__name__,id(logger)),flush=True)
    logger.warning('Warning raised by logger')

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
print('\nlogger without name: %s ID: %s'%(logger,id(logger)),flush=True)
logger.warning('Warning raised by logger')
logging_function()

logger = logging.getLogger(__name__)
print("\n--\nlogger %s called from name %s, ID: %s" %(logger,__name__,id(logger)),flush=True)
logger.warning('Warning raised by logger')
logging_function()



logger without name: <RootLogger root (DEBUG)> ID: 2014591529456


2022-05-24 17:44:44,644 WARNING Warning raised by logger



In Logging Function  <function logging_function at 0x000001D513B9AAF0>
logging:  <module 'logging' from 'c:\\Users\\kovor\\anaconda3\\envs\\folder_explorer\\lib\\logging\\__init__.py'>
logger without name: <RootLogger root (DEBUG)> ID: 2014591529456


2022-05-24 17:44:44,647 WARNING Warning raised by logger


logger <Logger __main__ (DEBUG)> called with name __main__, ID: 2014663619296


...

2022-05-24 17:44:44,649 WARNING Warning raised by logger



In Logging Function  <function logging_function at 0x000001D513B9AAF0>
logging:  <module 'logging' from 'c:\\Users\\kovor\\anaconda3\\envs\\folder_explorer\\lib\\logging\\__init__.py'>
logger without name: <RootLogger root (DEBUG)> ID: 2014591529456


2022-05-24 17:44:44,651 WARNING Warning raised by logger


logger <Logger __main__ (DEBUG)> called with name __main__, ID: 2014663619296


2022-05-24 17:44:44,652 WARNING Warning raised by logger


### What I can do with it

Quindi se creo due logger ``logger1`` e ``logger2``, creati rispettivamente con ``.getLogger()`` e ``.getLogger("__main__")`` il primo viene chiamato "root" ed è quello su cui viene scritto dai messaggi indirizzati sia a ``logging``, che a ``logger1``. I livelli di output sono pure selezionabili individualmente, ma le altre opzioni non competono ai logger, e sono solo settabili globalmente.

In [ ]:
import logging

#def test_current_function():
    
"""Test which logger is used when calling Logging."""

# does `getLogger`` alter linking?
print("\ncreate two loggers: ")

logger1 = logging.getLogger()
print('\n--\nlogger1 without name: %s ID: %s'%(logger1,id(logger1)))
logger1.setLevel(logging.INFO)

logger2 = logging.getLogger(__name__)
print("\n--\nlogger2 %s called from name %s, ID: %s" %(logger2,__name__,id(logger2)),flush=True)

logging.info('log written from logging.debug')
logger1.info('log written from logger1.debug')
logger2.info('log written from logger2.debug')

# does `logger.setlevel` alter linking?
print('\n--\nlogger1getlogger without name: %s ID: %s'%(logging.getLogger(),id(logging.getLogger())))

logger2.setLevel(logging.WARNING)
logging.info('log written from logging.debug')
logger1.info('log written from logger1.debug')
logger2.info('log written from logger2.debug')


create two loggers: 

--
logger1 without name: <RootLogger root (INFO)> ID: 2666684650832

--
logger2 <Logger __main__ (INFO)> called from name __main__, ID: 2666753372656


INFO:root:log written from logging.debug
INFO:root:log written from logger1.debug
INFO:__main__:log written from logger2.debug
INFO:root:log written from logging.debug
INFO:root:log written from logger1.debug



--
logger1getlogger without name: <RootLogger root (INFO)> ID: 2666684650832


What happens calling logger with names from functions?

In [ ]:
import logging
#sys.path.append('pyXTel\source\dataIO\\')
from logs import logging_function as lf
    

logger1 = logging.getLogger()
print('\n--\nlogger without name: %s ID: %s'%(logger1,id(logger1)))
print('handlers: %s (%i handlers)'%(logger1.handlers,len(logger1.handlers)),flush=True)

lf()

logger2 = logging.getLogger(__name__)
print("\n--\nlogger %s called from name %s, ID: %s" %(logger2,__name__,id(logger2)))
print('handlers: %s (%i handlers)'%(logger2.handlers,len(logger2.handlers)),flush=True)

lf()


--
logger without name: <RootLogger root (DEBUG)> ID: 2014591529456
handlers: [<StreamHandler stderr (NOTSET)>] (1 handlers)

In Logging Function  <function logging_function at 0x000001D513ECAD30>
logging:  <module 'logging' from 'c:\\Users\\kovor\\anaconda3\\envs\\folder_explorer\\lib\\logging\\__init__.py'>
logger without name: <RootLogger root (DEBUG)> ID: 2014591529456


2022-05-24 17:45:56,074 WARNING Warning raised by logger


logger <Logger logs (DEBUG)> called with name logs, ID: 2014663503248


2022-05-24 17:45:56,075 WARNING Warning raised by logger



--
logger <Logger __main__ (DEBUG)> called from name __main__, ID: 2014663619296
handlers: [] (0 handlers)

In Logging Function  <function logging_function at 0x000001D513ECAD30>
logging:  <module 'logging' from 'c:\\Users\\kovor\\anaconda3\\envs\\folder_explorer\\lib\\logging\\__init__.py'>
logger without name: <RootLogger root (DEBUG)> ID: 2014591529456


2022-05-24 17:45:56,077 WARNING Warning raised by logger


logger <Logger logs (DEBUG)> called with name logs, ID: 2014663503248


2022-05-24 17:45:56,079 WARNING Warning raised by logger


In [ ]:
logger1.handlers

[<StreamHandler stderr (NOTSET)>]

In questo modo pero', come fa ad intercettare log da funzioni interne che usano ``__name__``? A differenza dei print che vengono sempre intercettatti ``logging.getLogger(functionname)`` come fa a finire sui giusti handlers?

## Best practices and functions

# Advanced Use

## Modifying individual levels

Note that formatters applies to an entire handler or logger as a generic format for all levels. It is then not natively possible to set a different format for each level.



For example, next lines modify format of WARNING messages only (bold, red), works e.g. in IPython terminal. 

Il metodo qui presentato funziona, ma e' piuttosto permanente. 

Cio' e' douto al fatto che viene modificata l'etichetta del livello, cambiando il testo che la rappresenta. 

The solution uses``logging.addLevelName`` with a level which already have a name to replace the level name to a string including color code.

This is the string built from original names:

In [14]:
print("Original Name: %s"%logging.getLevelName(logging.ERROR))
print("Modified Name: \033[1;31m%s\033[1;0m" %logging.getLevelName(logging.ERROR))

Original Name: ERROR
Modified Name: ERROR


This has the effect to make the name global to the logger for all streamhandler (don't run the following, or run also the one after to undo)

In [41]:
# vedi ad esempio https://stackoverflow.com/questions/384076/how-can-i-color-python-logging-output
# ci sono parecchie soluzioni, per esempio:

#queste in effetti modificano i colori per output su terminale (e.g. iPython). Non funziona su file di testo.

#logging.basicConfig(filename='test_dev.log', level=logging.DEBUG)
logging.basicConfig(level=logging.DEBUG)
logging.addLevelName( logging.WARNING, "\033[1;31m%s\033[1;0m" % logging.getLevelName(logging.WARNING))
logging.addLevelName( logging.ERROR, "\033[1;41m%s\033[1;0m" % logging.getLevelName(logging.ERROR))

# dopo di che:

logging.info ('Start logging')
logging.error ('This error message is called with logging.error and no added formatting code')
logging.warning("This is a warning message")
logging.debug("This is a debug message")    

INFO:root:Start logging
ERROR:root:This error message is called with logging.error and no added formatting code


In [15]:
# undo previous cell
logging.addLevelName( logging.WARNING, "WARNING")
logging.addLevelName( logging.ERROR, "ERROR")


In [17]:
logger.debug('debug message')
logger.info('info message')
logger.warning('warn message')
logger.error('error message')
logger.critical('critical message')

DEBUG:root:debug message
INFO:root:info message
ERROR:root:error message
CRITICAL:root:critical message


In [16]:
# logging.addLevelName( logging.WARNING, "\033[1;31m%s\033[1;0m" % logging.getLevelName(logging.WARNING))

# logging.error("e' lo stesso che stampare direttamente la scritta EROR with color codes: :\n\033[1;41m%s\033[1;0m"%("EROR"))


Altro: come usarlo? ad esempio come faccio a loggare da un certo livello in su su file e altri livelli su schermo? Come faccio ad attivare/disattivare logging dello std output o logging su file? Diventa a questo punto possibile riusare i print nel programma.
Posso generare output testuale o html con diverse formattazioni per i diversi livelli?


## Enable/Disable capture of standard outptut

In [20]:
import logging
from logs import start_logger, reset_logger

In [23]:
#reset_logger(logger)
logger = start_logger('logging_tests.conf')
# import logging
# import logging.config

# logging.config.fileConfig('logging.conf')

# create logger
# logger = logging.getLogger('simpleExample')

# 'application' code
logger.debug('debug message')
logger.info('info message')
logger.warning('warn message')
logger.error('error message')
logger.critical('critical message')

N:  dataIO.logs
logger <RootLogger root (DEBUG)> existing
2022-05-23 20:03:28,280 - root - DEBUG - debug message
2022-05-23 20:03:28,280 - root - INFO - info message
2022-05-23 20:03:28,281 - root - WARNING - warn message
2022-05-23 20:03:28,282 - root - ERROR - error message
2022-05-23 20:03:28,283 - root - CRITICAL - critical message


In [24]:
logger

<RootLogger root (DEBUG)>

In [26]:
h=logger.handlers[0]

In [32]:
logging.warning('W!')

2022-05-23 20:09:43,829 - root - WARNING - W!


In [33]:
logger.warning('W!')

2022-05-23 20:10:01,084 - root - WARNING - W!


## Snapshots

In [18]:
from logging import config

In [6]:
h = logger.handlers[0]

In [7]:
h

<StreamHandler stderr (NOTSET)>

In [8]:
h.formatter

TypeError: format() missing 1 required positional argument: 'record'